In [421]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup,NavigableString
import numpy as np
import pandas as pd
from urllib.request import urlopen


### Create a Soup object from the Toronto Postal Code Url

In [422]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

### Create a DataFrame with the 'PostalCode','Borough','Neighborhood' columns

In [423]:
df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

### Scrap the postal code page and append the data in the DataFrame

In [424]:
data = []
table = soup.find('table', attrs={'rules':'all'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    for col in cols:
        postal_code=col.find('b').text
        a=col.find('a')
        if a:
            bourough=a.text
            if bourough:
                content=col.find('span').text.replace(')','')
                content=content.split('(')[1]
                neighborhood=content.replace('/',',')
                df = df.append({'PostalCode': postal_code,'Borough':bourough,'Neighborhood':neighborhood},ignore_index=True)

In [425]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Remove bad data

In [426]:
df = df[df.Borough != 'Business reply mail']

### Load the Geospatial csv file 

In [427]:
df1=pd.read_csv('Geospatial_Coordinates.csv')

### Rename the postal code column name to have the same name in the DataFrame df

In [428]:
df1.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True)

### Merge the postal code DataFrame with the Geospatial DataFrame use 'inner join'

In [429]:
df=df.merge(df1, how='inner')

In [430]:
df = df[df.Borough != 'Business reply mail']

In [431]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Save Result DataFrame into a CSV file

In [432]:
df.to_csv('postalcode.csv',index=False)